## 批量归一化
本节我们介绍批量归一化$（batch normalization）$层，它能让较深的神经网络的训练变得更加容易。在$3.16$节（实战$Kaggle$比赛：预测房价）里，我们对输入数据做了标准化处理：处理后的任意一个特征在数据集中所有样本上的均值为$0$、标准差为$1$。标准化处理输入数据使各个特征的分布相近：这往往更容易训练出有效的模型。

通常来说，**数据标准化预处理对于浅层模型就足够有效了**。随着模型训练的进行，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经网络来说，**即使输入数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化**。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。**批量归一化和下一节将要介绍的残差网络为训练和设计深度模型提供了两类重要思路**。

### 批量归一化层
对全连接层和卷积层做批量归一化的方法稍有不同。下面我们将分别介绍这两种情况下的批量归一化。

### 对全连接层做批量归一化
我们先考虑如何对全连接层做批量归一化。通常，我们将批量归一化层置于全连接层中的仿射变换和激活函数之间。设全连接层的输入为$u$，权重参数和偏差参数分别为$W$和$b$，激活函数为$\phi$。设批量归一化的运算符为$BN$。那么，使用批量归一化的全连接层的输出为
<center>
    $\phi(BN(\mathbf{x}))$
</center>

其中批量归一化输入$\mathbf{x}$由仿射变换
<center>
    $\mathbf{x}=\mathbf{W}\mathbf{u}+\mathbf{b}$
</center>

得到。考虑一个由m个样本组成的小批量，仿射变换的输出为一个新的小批量$\mathbf{\beta}={\mathbf{x}^{(1)},...,\mathbf{x}^{(m)}}$。它们正是批量归一化层的输入。对于小批量$\mathbf{\beta}$中任意样本$\mathbf{x}^{(i)}\epsilon\mathbb{R}^{d},1\le i\le m$，批量归一化层的输出同样是$d$维向量
<center>
    $ \mathbf{y}^{(i)}=\mathbf{B}\mathbf{N}(\mathbf{x}^{(i)}) $
</center>

并由以下几步求得。首先，对小批量$\mathbf{\beta}$求均值和方差
<center>
    $ \mathbf{u}_{\beta}\leftarrow\frac{1}{m}\sum_{i=1}^m\mathbf{x}^{(i)} $
</center>
<center>
    $ \mathbf{\sigma}_{\beta}^2\leftarrow\frac{1}{m}\sum_{i=1}^m(\mathbf{x}^{(i)}-\mathbf{u}_{\beta})^2 $
</center>

其中的平方计算是按元素求平方。接下来，使用按元素开放和按元素除法对$\mathbf{x}^{(i)}$标准化：
<center>
    $ \hat{\mathbf{x}}^{(i)}\leftarrow\frac{\mathbf{x}^{(i)}-\mathbf{u}_{\beta}}{\sqrt{\sigma_{\mathbf{\beta}}^2+\epsilon}} $
</center>

这里$\epsilon>0$是一个很小的常数，保证分母大于$0$。在上面标准化的基础上，批量归一化层引入了两个可以学习的模型参数，拉伸$(scale)$参数$\mathbf{\gamma}$和偏移$(shift)$参数$\mathbf{\beta}$。这两个参数和$\mathbf{x}^{(i)}$形状相同，皆为$d$维向量。他们与$\mathbf{x}^{(i)}$分别做按元素乘法（符号$\bigodot$）和加法运算:
<center>
    $ \mathbf{y}^{(i)}\leftarrow\mathbf{\gamma}\bigodot\hat{\mathbf{x}}^{(i)}+\mathbf{\beta} $
</center>

至此，我们得到了$\mathbf{x}^{(i)}$的批量归一化的输出$\mathbf{y}^{(i)}$。值得注意的是，可学习的拉伸和偏移参数保留了不对$\hat{\mathbf{x}}^{(i)}$做批量归一化的可能：此时只需学出$\mathbf{\gamma}=\sqrt{\mathbf{\sigma}_{\mathbf{\beta}}^2+\epsilon}$和$\mathbf{\beta}=\mathbf{u}_{\mathbf{\beta}}$。我们可以对此这样理解：**如果批量归一化无益，理论上，学出的模型可以不使用批量归一化**。

### 对卷积层做批量归一化
对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且**每个通道都拥有独立的拉伸和偏移参数，并均为标量**。设小批量中有$m$个样本。在单个通道上，假设卷积计算输出的高和宽分别为$p$和$q$。我们需要对该通道中$m\times p\times q$个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中$m\times p\times q$个元素的均值和方差。

### 预测时的批量归一化
使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。**可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的**。

### 从零开始实现
下面我们自己实现批量归一化层。

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    if not is_training:
        X_hat = (X - moving_mean)/torch.sqrt(moving_var+eps)
    else:
        assert len(X.shape) in (2,4)
        if len(X.shape) == 2:
            mean = X.mean(dim=0)
            var = ((X - mean)**2).mean(dim=0)
        else:
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean)**2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        X_hat = (X - mean)/torch.sqrt(var+eps)
        moving_mean = momentum*moving_mean+(1.0-momentum)*mean
        moving_var = momentum*moving_var+(1.0-momentum)*var
    Y = gamma*X_hat+beta
    return Y, moving_mean, moving_var

接下来，我们自定义一个$BatchNorm$层。它保存参与求梯度和迭代的拉伸参数$gamma$和偏移参数$beta$，同时也维护移动平均得到的均值和方差，以便能够在模型预测时被使用。$BatchNorm$实例所需指定的$num\_features$参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数。该实例所需指定的$num\_dims$参数对于全连接层和卷积层来说分别为$2$和$4$。

In [2]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)
        
    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        Y, self.moving_mean, self.moving_var = batch_norm(self.training, 
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

### 使用批量归一化层的LeNet
下面我们修改$5.5$节（卷积神经网络（$LeNet$））介绍的$LeNet$模型，从而应用批量归一化层。我们在所有的卷积层或全连接层之后、激活层之前加入批量归一化层。

In [3]:
net = nn.Sequential(
    nn.Conv2d(1, 6, 5),
    BatchNorm(6, num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2),
    nn.Conv2d(6, 16, 5),
    BatchNorm(16, num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2),
    d2l.FlattenLayer(),
    nn.Linear(16*4*4, 120),
    BatchNorm(120, num_dims=2),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    BatchNorm(84, num_dims=2),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)

下面我们训练修改后的模型。

In [4]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size,root='~/wms/Jupyter/Datasets/FashionMNIST')

lr, epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, epochs)

training on: cuda:7
epoch 1, loss 0.9842, train acc 0.788, test acc 0.815, time 3.5 sec
epoch 2, loss 0.4610, train acc 0.862, test acc 0.846, time 3.1 sec
epoch 3, loss 0.3697, train acc 0.877, test acc 0.815, time 3.2 sec
epoch 4, loss 0.3342, train acc 0.885, test acc 0.865, time 3.5 sec
epoch 5, loss 0.3088, train acc 0.891, test acc 0.869, time 2.9 sec


最后我们查看第一个批量归一化层学习到的拉伸参数$gamma$和偏移参数$beta$。

In [5]:
net[1].gamma.view((-1,)), net[1].beta.view((-1,))

(tensor([1.2680, 0.8549, 0.9330, 0.7769, 1.1262, 1.1766], device='cuda:7',
        grad_fn=<ViewBackward>),
 tensor([ 0.3031, -0.2546, -0.5678, -0.2099, -0.5759,  0.2490], device='cuda:7',
        grad_fn=<ViewBackward>))

### 简洁实现
与我们刚刚自己定义的$BatchNorm$类相比，$Pytorch$中$nn$模块定义的$BatchNorm1d$和$BatchNorm2d$类使用起来更加简单，二者分别用于全连接层和卷积层，都需要指定输入的$num\_features$参数值。下面我们用$PyTorch$实现使用批量归一化的$LeNet$。

In [6]:
net = nn.Sequential(
    nn.Conv2d(1, 6, 5),
    nn.BatchNorm2d(6),
    nn.Sigmoid(),
    nn.MaxPool2d(2),
    nn.Conv2d(6, 16, 5),
    nn.BatchNorm2d(16),
    nn.Sigmoid(),
    nn.MaxPool2d(2),
    d2l.FlattenLayer(),
    nn.Linear(16*4*4, 120),
    nn.BatchNorm1d(120),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    nn.BatchNorm1d(84),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [7]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size,root='~/wms/Jupyter/Datasets/FashionMNIST')

lr, epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, epochs)

training on: cuda:7
epoch 1, loss 0.9819, train acc 0.794, test acc 0.817, time 2.7 sec
epoch 2, loss 0.4498, train acc 0.866, test acc 0.842, time 2.7 sec
epoch 3, loss 0.3597, train acc 0.880, test acc 0.872, time 2.7 sec
epoch 4, loss 0.3257, train acc 0.888, test acc 0.878, time 2.8 sec
epoch 5, loss 0.3002, train acc 0.895, test acc 0.883, time 2.5 sec


+ 在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络的中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。
+ 对全连接层和卷积层做批量归一化的方法稍有不同。
+ 批量归一化层和丢弃层一样，在训练模式和预测模式的计算结果是不一样的。
+ PyTorch提供了BatchNorm类方便使用。